In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp = requests.get(url)

In [4]:
resp.encoding = 'utf-8'    #轉換編碼至UTF-8
resp.encoding = 'big5'
resp.status_code

200

In [30]:
soup = BeautifulSoup(resp.text, 'lxml')

In [7]:
table = soup.find('table', {'class': 'wikitable sortable'})

In [8]:
columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
columns

['Postcode', 'Borough', 'Neighbourhood']

In [9]:
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
rows[:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront']]

In [12]:
import pandas as pd
df = pd.DataFrame(data=rows, columns=columns)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
df1 = df[~df['Borough'].isin(['Not assigned'])]
#df2=df1.groupby("Postcode").agg(lambda x:','.join(set(x)))
df2 = df1.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [26]:
mask = df2['Neighbourhood'] == "Not assigned"
df2.loc[mask, 'Neighbourhood'] = df2.loc[mask, 'Borough']
df2.shape

(103, 3)

In [29]:
!wget -O loan_train.csv http://cocl.us/Geospatial_data

--2020-03-12 08:17:44--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 169.48.113.194, 158.85.108.83
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-03-12 08:17:46--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-12 08:17:47--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-03-12 08:17:47--  https://ibm.box.com/publ

In [36]:
dff = pd.read_csv('loan_train.csv')
dff.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
dff.rename(index=str, columns={"Postal Code": "Postcode"}, inplace = True)
neighborhood = pd.merge(df2, dff, on="Postcode", how='inner')

In [64]:
!conda install folium -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTAL

In [66]:
!conda list

# packages in environment at /opt/conda/envs/Python36:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_py-xgboost-mutex         2.0                       cpu_0  
absl-py                   0.7.0                    py36_0  
alabaster                 0.7.12                     py_0  
anaconda-client           1.7.2                    py36_0  
anaconda-project          0.8.2                    py36_0  
arcgis                    1.6.0            py36h39e3cac_1    esri
arrow-cpp                 0.11.1           py36h5c3f529_1  
asn1crypto                0.24.0                   py36_0  
astor                     0.7.1                    py36_0  
astroid                   2.1.0                    py36_0  
astropy                   3.1.1            py36h7b6447c_0  
astunparse                1.6.2                     <pip>
atomicwrites              1.3.0                    py36_1  
attrs                     18.2.0

In [85]:
neighborhood.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,0,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [84]:
neighborhood.insert(0, 'Cluster Labels', 0)

In [86]:
import folium
import numpy as np
from matplotlib import cm
from matplotlib import colors
map_clusters = folium.Map(neighborhood=['Latitude', 'Longitude'], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood['Latitude'], neighborhood['Longitude'], neighborhood['Neighbourhood'], neighborhood['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters